In [1]:
import os
import pandas as pd
import faiss
import pickle
from sentence_transformers import SentenceTransformer
import openai
import numpy as np


In [1]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
df = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6022 entries, 0 to 6021
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   분류번호    6022 non-null   object
 1   사무명     6022 non-null   object
 2   사무개요    6022 non-null   object
 3   신청유형    6022 non-null   object
 4   신청자 자격  6020 non-null   object
 5   수수료     6021 non-null   object
 6   수수료내역   6018 non-null   object
 7   소관부처    6022 non-null   object
 8   담당부서    6022 non-null   object
 9   접수방법    6022 non-null   object
 10  접수/처리   6013 non-null   object
 11  근거법령    5981 non-null   object
 12  신청서     5124 non-null   object
 13  발급물 내용  918 non-null    object
 14  구비서류    5372 non-null   object
dtypes: object(15)
memory usage: 705.8+ KB


In [9]:
df.head(20)

,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류
0,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN
1,9740000-0009,공탁신청(공탁물지급청구 포함),"공탁 신청, 공탁금의 지급을 위한 민원입니다. 공탁은 공탁자가 법령에 규정된 원인에...",승인,누구나신청가능,수수료 없음,수수료 없음,대법원,사법등기국,"방문, 인터넷",1. 유형없음(총 0일)\n*접수:지방법원.지원(0일)\n*경유:\n*협의:\n*처...,"1. 공탁법(제4조, 제9조) \n2. 공탁규칙(제20조, 제21조, 제23조, 제...",금전공탁서(변제 등) 공탁사무 문서양식에 관한 예규 : 별지서식 1호의 1\n,NaN,"1. 유형없음\n< 민원인 제출서류 >\n(1) 인감증명서 신분증 가압류,압류결정문..."
2,9740000-0008,친양자입양취소신고,가정법원의 허가를 얻어 친양자 입양을 취소하기 위해 신청하는 민원입니다.,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 즉시)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처리...,"1. 가족관계의 등록 등에 관한 법률(제69조, 제70조)","제9호, 친양자 입양취소신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자입양취소당사자의 가족관계등록부...
3,9740000-0007,친양자파양신고,가정법원의 허가를 얻어 친양자입양을 파양하고자 하는 사람이 재판의 확정일부터 1개월...,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 지체없이)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*...,1. 가족관계의 등록 등에 관한 법률(제69조),"제7호, 친양자파양신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 친양자파양당사자의 가족관계등록부의 ...
4,9740000-0006,성·본계속사용신고,부가 혼인외 자를 인지할 때 인지 전의 자녀의 성과 본을 유지할 경우 그 취지를 가...,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음(총 0일)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*처...,1. 가족관계의 등록 등에 관한 법률(제55조 제1항),"제3호, 성·본계속사용신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 성·본계속사용자의 가족관계등록부의 ...
5,9740000-0005,성·본변경신고,자의 복리를 위하여 가정법원의 허가를 얻어 자녀의 성.본을 변경하고자 하는 자가 재...,신고,누구나신청가능,수수료 없음,수수료 없음,대법원,가족관계등록과,"방문, 우편",1. 유형없음( 지체없이)\n*접수:시.구.읍.면(0일)\n*경유:\n*협의:\n*...,1. 가족관계의 등록 등에 관한 법률(제100조 제1항),"제34호, 성·본변경신고서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 성본변경신고서\n(2) 가정법원의 ...
6,9740000-0004,가족관계등록부등의 증명서 발급,"국민 개개인의 출생, 입양, 혼인, 사망 등 가족관계의 발생 및 변동에 관한 증명서...",증명,본인또는대리인신청,사례별 수수료,"증명서 1통당 1,000원 / 인터넷으로 발급받을 때는 무료임",대법원,가족관계등록과,"방문, 인터넷, 무인발급기","1. 유형없음( 즉시)\n*접수:동, 시.구.읍.면(0일)\n*경유:\n*협의:\n...",1. 가족관계의 등록 등에 관한 법률(제14조),"별지 제11호 서식, 가족관계 등록사항별 증명서 교부 등 신청서\n",NaN,1. 유형없음\n< 민원인 제출서류 >\n(1) 신분증 지참
7,9740000-0003,건물등기사항증명서 발급,건물에 관한 등기기록 사항의 전부 또는 일부를 증명하는 서면을 발급받기 위한 민원입...,증명,누구나신청가능,사례별 수수료,"방문신청 통당 1,200원/무인발급기 및 인터넷발급신청 통당 1,000원",대법원,부동산등기과,"방문, 인터넷, 무인발급기",1. 유형없음(총 0일)\n*접수:지방등기소(0일)\n*경유:\n*협의:\n*처리:...,NaN,NaN,NaN,NaN
8,9740000-0002,토지등기사항증명서 발급,토지에 관한 등기기록 사항의 전부 또는 일부를 증명하는 서면을 발급받기 위한 민원입...,증명,누구나신청가능,사례별 수수료,"방문신청 통당 1,200원/무인발급기 및 인터넷발급신청 통당 1,000원",대법원,부동산등기과,"방문, 인터넷, 무인발급기",1. 유형없음(총 0일)\n*접수:지방등기소(0일)\n*경유:\n*협의:\n*처리:...,NaN,NaN,NaN,NaN
9,9740000-0001,법인등기사항증명서 발급,"법인등기사항증명서를 발급받기 위한 민원입니다. 법인등기사항증명서는 법인(상사법인, ...",교부,누구나신청가능,사례별 수수료,"방문신청 통당 1,200원/무인발급기 및 인터넷발급신청 통당 1,000원",대법원,사법등기국,"방문, 인터넷",1. 유형없음(총 0일)\n*접수:지방등기소(0일)\n*경유:\n*협의:\n*처리:...,1. 상업등기법(제15조) \n2. 상업등기규칙(제26조),NaN,NaN,NaN


In [10]:
#담당부서 unique 몇개인지 찾는식
unique_departments = df['담당부서'].unique()
unique_department1s = df['소관부처'].unique()
print(f"Unique departments: {len(unique_departments)}")
print(f"Unique departments: {len(unique_department1s)}")

Unique departments: 753
Unique departments: 46


In [4]:
# 청크 생성
documents = []
metadatas = []
for idx, row in df.iterrows():
    chunk = "\n".join([f"[{col}] {row[col]}" for col in df.columns])
    documents.append(chunk)
    metadatas.append({"사무명": row["사무명"], "분류번호": row["분류번호"]})


In [5]:
# # 임베딩 생성
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# roberta 모델을 사용하여 문서 임베딩 생성 -> 출력 차원: 768

#각 청





In [21]:
embeddings = model.encode(documents, show_progress_bar=True)
# # FAISS 저장
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# # 인덱스와 메타데이터 저장
faiss.write_index(index, "rag_index.faiss")
with open("rag_metadata.pkl", "wb") as f:
    pickle.dump((documents, metadatas), f)

Batches:   0%|          | 0/189 [00:00<?, ?it/s]

In [22]:
# 인덱스 및 메타데이터 로드
index = faiss.read_index("rag_index.faiss")
with open("rag_metadata.pkl", "rb") as f:
    documents, metadatas = pickle.load(f)

In [41]:
from openai import OpenAI
client = OpenAI()

In [ ]:
# 검색 함수
def search_similar_documents(query, top_k=3):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, top_k)
    return [(documents[i], metadatas[i]) for i in I[0]]

# 프롬프트 생성
def build_prompt(query, retrieved_docs):
    context = "\n\n".join([doc for doc, meta in retrieved_docs])
    return f"""다음은 민원 관련 문서입니다:\n\n{context}\n\n질문: {query}\n\n답변:"""

# 사용자 입력 → GPT 응답 생성
def answer_query(query):
    retrieved_docs = search_similar_documents(query)
    prompt = build_prompt(query, retrieved_docs)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 민원 정보를 잘 알려주는 공공서비스 상담 챗봇입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content

In [43]:
# 예시 사용
query = "건축물 대장을 발급받으려면 어떻게 해야 하나요?"
response = answer_query(query)
print(response)

건축물 대장을 발급받기 위해서는 다음과 같은 절차를 따르시면 됩니다.

1. 신청 유형: 교부
2. 신청자 자격: 누구나 신청 가능
3. 수수료: 발급(1건당):500원, 열람(1건당):300원, 인터넷 발급(열람)시 무료
4. 접수 방법: 방문, 우편, FAX, 인터넷, 모바일, 무인발급기
5. 접수/처리: 시군구 및 읍면동 출장소에서 접수하며, 처리는 즉시 이루어집니다.
6. 신청서: 건축물대장등.초본발급 및 열람신청서를 작성하셔야 합니다.

발급물 내용으로는 일반건축물대장, 집합건축물대장(표제부), 집합건축물대장(전유부), 건축물대장 총괄 등이 있습니다. 

참고로, 건축물 대장은 건축물의 위치, 구조, 용도, 면적 등 건축물에 관한 중요한 사항을 기록한 공식 문서입니다.


In [ ]:
# GPT 응답 생성
def generate_answer(query):
    retrieved = search_similar_documents(query)
    context = "\n\n".join([doc for doc, meta in retrieved])
    prompt = f"""다음은 민원 관련 문서입니다:\n\n{context}\n\n질문: {query}\n\n답변:"""

    response = openai.ChatCompletion.create(
        model="gpt-4",  # 또는 "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "당신은 민원 정보를 잘 알려주는 공공기관 상담 챗봇입니다. "},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return response['choices'][0]['message']['content']


In [47]:
import streamlit as st
# Streamlit UI
st.title("📄 민원 상담 GPT 챗봇")
query = st.text_input("궁금한 민원 내용을 입력해주세요:")

if st.button("답변 받기") and query:
    with st.spinner("GPT가 답변을 생성 중입니다..."):
        answer = generate_answer(query)
        st.success(answer)

2025-04-30 23:25:01.712 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.752 
  command:

    streamlit run /Users/type/textmining-pj/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-04-30 23:25:01.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-30 23:25:01.753 Sessi

NameError: name 'index' is not defined